Реализуем градиентный спуск для задачи поиска оптимальных коэффициентов в MSE регрессии!

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns

Имеем 1000 объектов и 10 признаков у каждого (+таргет)!

Обучим модель линейной регрессии:

$$
a(x) = \beta_1 d_{1} + \beta_2 d_{2} + \beta_3 d_{3} + \beta_4 d_{4} + \beta_5 d_{5} + \beta_6 d_{6} + \beta_7 d_{7} + \beta_8 d_{8} + \beta_9 d_{9} + \beta_{10} d_{10} + \beta_0
$$

Которая минимизирует MSE:

$$
Q(a(X), Y) = \sum_i^{1000} (a(x_i) - y_i)^2
$$

In [3]:
data = pd.read_csv('gradient_descent.csv')

X = data.drop('target', axis=1)
Y = pd.DataFrame({'target':data['target']})


model = LinearRegression()
model.fit(X, Y)
X.shape

(1000, 10)

Обучим коэффициенты линейной регрессии с помощью библиотеки <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html"> **sklearn** </a>

Отдельно выведем оценку свободного коэффициента  ($\beta_0$ при $d_0 = 1$)

In [4]:
Y.head(1)

,target
0,30.147094


Теперь вам необходимо реализовать класс для оптимизации коэффициентов линейной регрессии МНК.
Подразумевается, что на вход алгоритм будет принимать следующие параметры:

- 2 pandas датафрейма **samples** и **targets**, содержащих матрицу объектов и ветор ответов соответственно
- значение **learning rate**, который корректирует длину вектора-градиента (чтобы он не взорвался)
- значение **threshold**'а для критерия останова (когда мы считаем, что мы сошлись к оптимуму)
- параметр **copy**, который позволяет либо делать изменения in-place в датафрейме, подающимся в класс, если изменения матрицы объектов в принципе при обучении имеются. Или же копировать объект при инициализации класса и возвращать новый объект, если требуется.

Он будет состоять из следующих важных компонент-методов:

- **add_constant_feature**: добавляет колонку с названием *constant* из единичек к переданному датафрейму **samples**. Это позволяет оценить свободный коэффициент $\beta_0$.

- **calculate_mse_loss**: вычисляет при текущих весах **self.beta** значение среднеквадратической ошибки.

- **calculate_gradient**: вычисляет при текущих весах вектор-градиент по функционалу.

- **iteration**: производит итерацию градиентного спуска, то есть обновляет веса модели, в соответствии с установленным **learning_rate = $\eta$**: $\beta^{(n+1)} = \beta^{(n)} - \eta \cdot \nabla Q(\beta^{(n)})$

- **learn**: производит итерации обучения до того момента, пока не сработает критерий останова обучения. В этот раз критерием останова будет следующее событие: во время крайней итерации изменение в функционале качества модели составило значение меньшее, чем **self.threshold**. Иными словами, $|Q(\beta^{(n)}) - Q(\beta^{(n+1)})| < threshold$.

P.S. установите в **__init__** аттрибут экземпляра с названием **iteration_loss_dict**, который будет устроен следующим образом: на каждой итерации мы будем добавлять в словарь пару ключ-значение, где ключем будет номер итерации $n$, а значением - среднеквадратическая ошибка в точке $\beta^{(n)}$. Это пригодится нам в будущем для визуализации.

### Hint: пример вычисления производной

$$
Q(a, X) = \frac{1}{N}\cdot\sum_{i=1}^N (\beta_1 \cdot d_{i1} + ... + \beta_n \cdot d_{in} - y_i)^2
$$

Выше - минимизируемая функция. Она зависит от n переменных: $\beta_1, ..., \beta_n$. Вектор-градиент - матрица с одной строчкой, состоящей из производных 1го порядка по всем переменным.

$$
\nabla Q(a, X) = (Q'_{\beta_1} \;\;\; Q'_{\beta_2} \;\;\; ... \;\;\; Q'_{\beta_{n-1}}  \;\;\;  Q'_{\beta_n})
$$

Пример вычисления производной по первой переменной:

$$
Q'_{\beta_1} = \frac{2}{N} \cdot \sum_{i=1}^N d_{i1} (\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i)
$$

Скажем, для нашего датасета X, Y вычислим эту саму производную при начальных единичных коэффициентах $\beta_{start} = (1 \;\;\; 1 \;\;\; ...)$

Получим для каждого объекта в начале выражение из скобочек: 
$$
\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i
$$

In [5]:
### Инициализируем точку для коэффициентов в виде вектора из единичек
initial_betas = np.ones(X.shape[1])

### Получим выражение выше для каждого объекта. 
### Для этого скалярно перемножим строчки из X на наши beta

scalar_value = np.dot(X, initial_betas.reshape(-1, 1)).ravel()
print(scalar_value)

[4.24339854 3.9524118  5.16354431 5.05108987 3.70336585 5.97562576
 5.36645618 4.64371227 5.31215445 5.13559383 4.57902419 4.32294422
 5.54997113 3.38646272 5.46648432 4.15761811 4.54622922 4.92460189
 5.49512761 4.20534235 3.79440676 3.64754149 3.3861493  5.28347287
 4.63170572 3.82066088 5.70447535 5.00305454 5.18649487 4.32348734
 5.37919592 5.30162516 4.73808802 4.58758426 5.64623264 3.92151158
 5.52622189 4.56147026 3.64504638 5.78092671 4.37963386 3.73847398
 4.56174807 5.42891712 5.33241798 4.92249654 4.00705898 4.96133463
 4.00952333 3.54853568 5.57722518 3.6827849  4.36082279 5.8455649
 4.66986831 6.65566972 4.363927   3.24811339 4.12653761 4.29553237
 4.10798391 4.87482995 5.62744705 4.99671407 4.75811197 4.45837844
 6.02034933 2.88183343 4.30169734 6.13075889 6.25145428 5.46300566
 5.09670393 3.39861176 4.45886785 4.42007817 6.14577431 5.96881144
 5.27359872 4.32991041 6.64957318 5.67765577 4.74732862 5.47829417
 5.92707771 5.91747901 3.20068905 6.20800637 4.72749617 4.71983

Теперь полученное значение для каждого объекта умножим на соответствующее значение признака $d_1$:

$$
d_{i1} \cdot (\beta_1 \cdot d_{i1} + ... + \beta_{n} \cdot d_{in} - y_i)
$$

In [6]:
### Возьмем столбик со значениями 1 признака

d_i1 = X.values[:, 0]
### Умножим каждый объект на соответствующее значение признака
scalar_value = scalar_value * d_i1

In [7]:
### Наконец, умножим все на 2 и усреднимся, 
### чтобы получить значение производной по первому параметру

2 * np.mean(scalar_value)

5.30115305614268

### Эта логика поможем Вам при реализации класса!

learn(self)

метод возвращает итоговую среднеквадратическую ошибку.
метод итеративно вычисляет среднеквадратическую ошибку и вектор-градиент. номер итерации и MSE записываются в словарь *iteration_loss_dict*. критерий останова срабатывает тогда, когда абсолютное значение разницы двух последних MSE меньше *self.threshold*.

In [8]:

class GradientDescentMse:
    """
    Базовый класс для реализации градиентного спуска в задаче линейной МНК регрессии
    """

    def __init__(self, samples: pd.DataFrame, targets: pd.DataFrame,
                 learning_rate: float = 1e-3, threshold=1e-6, copy: bool = True):
        """
        self.samples - матрица признаков
        self.targets - вектор таргетов
        self.beta - вектор из изначальными весами модели == коэффициентами бета (состоит из единиц)
        self.learning_rate - параметр *learning_rate* для корректировки нормы градиента
        self.threshold - величина, меньше которой изменение в loss-функции означает остановку градиентного спуска
        iteration_loss_dict - словарь, который будет хранить номер итерации и соответствующую MSE
        copy: копирование матрицы признаков или создание изменения in-place
        """
        self.samples = samples
        self.targets = targets
        self.beta = np.ones(self.samples.shape[1])
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.iteration_loss_dict = {}
        self.copy = copy


    def add_constant_feature(self):
        """
        Метод для создания константной фичи в матрице объектов samples
        Метод создает колонку с константным признаком (interсept) в матрице признаков.
        Hint: так как количество признаков увеличилось на одну, не забудьте дополнить вектор с изначальными весами модели!
        """
        self.samples['constant'] = 1

        self.beta = np.ones(self.samples.shape[1])

        if not self.copy:
            return

    def calculate_mse_loss(self) -> float:
        """
        Метод для расчета среднеквадратической ошибки

        :return: среднеквадратическая ошибка при текущих весах модели : float
        """
        predicted = np.dot(self.samples, self.beta)

        mse = np.mean((predicted - self.targets) ** 2)
        return mse

    def calculate_gradient(self) -> np.ndarray:
        """
        Метод для вычисления вектора-градиента
        Метод возвращает вектор-градиент, содержащий производные по каждому признаку.
        Сначала матрица признаков скалярно перемножается на вектор self.beta, и из каждой колонки
        полученной матрицы вычитается вектор таргетов. Затем полученная матрица скалярно умножается на матрицу признаков.
        Наконец, итоговая матрица умножается на 2 и усредняется по каждому признаку.

        :return: вектор-градиент, т.е. массив, содержащий соответствующее количество производных по каждой переменной : np.ndarray
        """
        a = np.dot(self.samples, self.beta)
        b = a - self.targets
        ans = (np.dot(b, self.samples) * 2) / len(self.samples)
        return ans

    def iteration(self):
        """
        Обновляем веса модели в соответствии с текущим вектором-градиентом
        """
        b = self.calculate_gradient()
        self.beta -= self.learning_rate * b

    def learn(self):
        """
        Итеративное обучение весов модели до срабатывания критерия останова
        Запись mse и номера итерации в iteration_loss_dict

        Описание алгоритма работы для изменения бет:
            Фиксируем текущие beta -> start_betas
            Делаем шаг градиентного спуска
            Записываем новые beta -> new_betas
            Пока |L(new_beta) - L(start_beta)| > threshold:
                Повторяем первые 3 шага

        Описание алгоритма работы для изменения функции потерь:
            Фиксируем текущие mse -> previous_mse
            Делаем шаг градиентного спуска
            Записываем новые mse -> next_mse
            Пока |(previous_mse) - (next_mse)| > threshold:
                Повторяем первые 3 шага
        """
        previous_mse = self.calculate_mse_loss()
        self.iter_count = 0
        while True:
            self.iteration()
            next_mse = self.calculate_mse_loss()

            self.iteration_loss_dict[self.iter_count] = next_mse

            if abs(previous_mse - next_mse) < self.threshold:
                break

            previous_mse = next_mse
            self.iter_count += 1


Обучим коэффициенты линейной модели с помощью реализованного нами градиентного спуска, не забыв добавить свободную переменную. Получились ли такие же коэффициенты, как и при использовании **LinearRegression** из **sklearn**? Если нет, то почему они отличаются, на Ваш взгляд, и сильно ли?

In [13]:
GD = GradientDescentMse(samples=X, targets=Y)
GD.add_constant_feature()
GD.learn()
print(GD.beta)
print(GD.calculate_mse_loss())

[0.88733305 1.90100713 2.88063607 3.87662612 4.89623507 5.89126182
 6.89254811 7.90311947 8.87580109 9.86109585 4.94854733]
0.010732740689782506


In [459]:
print(answers)

{'0.01, 0.1': [0.42914450820919203, 98], '0.01, 0.05': [0.7933106723839498, 152], '0.01, 0.01': [2.3084563042218504, 271], '0.01, 0.005': [3.317604378398448, 162], '0.01, 0.001': [3.4188440110378284, 502], '0.001, 0.1': [0.13930745981563908, 200], '0.001, 0.05': [0.04191600215652436, 324], '0.001, 0.01': [0.44096699878774426, 988], '0.001, 0.005': [0.8060077108681554, 1522], '0.001, 0.001': [2.3139393273900146, 2709], '0.0001, 0.1': [0.11244969609548172, 417], '0.0001, 0.05': [0.14758909576416868, 688], '0.0001, 0.01': [0.13561442389653577, 2009], '0.0001, 0.005': [0.03803198592345847, 3244], '0.0001, 0.001': [0.4430542743091843, 9881], '1e-05, 0.1': [0.037352129147760205, 664], '1e-05, 0.05': [0.05278662720066507, 1180], '1e-05, 0.01': [0.11264763801782673, 4179], '1e-05, 0.005': [0.14751404098538867, 6888], '1e-05, 0.001': [0.13525905667795768, 20100]}


In [ ]:
print('Веса модели при переменных d1, d2, ..., d10 равны соответственно: \n\n' + str(GD.beta))
print(GD.iteration_loss_dict)

In [ ]:
### Укажите в ответе на задание 2 оценку свободного коэффициента

Попробуйте теперь изменить значения **learning_rate** и/или **threshold**. Например, установите длину шага $\eta = 1$. Что произошло и почему такое возможно?

In [ ]:
### Your code is here

В машинном обучении зачастую исследуют так называемые **траектории обучения** (или **learning paths**). Это графики, показывающие, как во время обучения при каждой следующей итерации изменялось значение минимизируемого функционала. Постройте такие траектории для различных **learning rate**'ов и **threshold**'ов. Советуем использовать для этого разобранный на занятиях **add_subplot** метод. 

Возьмите следующие **threshold**'ы: 1e-2, 1e-3, 1e-4, 1e-5

И следующие значения **learning rate**'а: 1e-1, 5e-2, 1e-2, 5e-3, 1e-3

У вас должен получиться примерно такой график (см. ниже, значения среднеквадратической ошибки мы намеренно замазали оранжевыми квадратиками, чтобы не спойлерить вам результаты).

Как и подобает хорошим Data Scientist'ам, не забывайте подписывать графики, оси, а так же делать элементы ваших визуализаций читаемыми и видимыми. Советуем пересмотреть методы и параметры форматирования из лекции.

При какой комбинации **threshold** - **learning rate** из возможных предложенных выше, получается достигнуть меньшего значения нашей минимизируемой функции? Запишите каждой из значений в легенде на графиках.


In [461]:
fig = plt.figure()

fig.set_size_inches(13, 10)

"""Задать списки для итераций"""
### Your code is here 

for i in range(len(thresholds)):
    """Задать threshold, добавить график на полонто, создать список для хранения значений функционала"""
    ### Your code is here
    
    for lr in rates:
        """Создать объект модели, добавить константу, запустить обучение модели"""
        ### Your code is here
        
        """Определить learning_path через атрибут iteration_loss_dict"""
        ### Your code is here
        
        """Отобразить learning_path на графике"""
        ### Your code is here
        plt.ylim(0, 100)
        plt.xlim(0, 2000)
        
        Q_values.append(str(round(list(learning_path.values())[-1], ndigits=4)))
    
    plt.ylabel('Среднеквадратическая ошибка')
    plt.xlabel('Номер итерации')
    plt.legend([f'Learning rate equals to {rates[i]}' + ' with Q = ' + Q_values[i] for i in range(len(rates))])

fig.tight_layout() 

plt.show()

NameError: name 'thresholds' is not defined

<Figure size 1300x1000 with 0 Axes>

In [ ]:
### Укажите в ответе на задание 3 значения threshold и learning rate через запятую и пробел 
### (при которых достигается меньшее значение минимизируемой функции)